In [ ]:
!pip install cloudscraper pandas selenium webdriver-manager chromedriver-autoinstaller


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y wget gnupg
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
!apt-get update
!apt-get install -y google-chrome-stable


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [969 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,116 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Hit:13 https://p

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
import random

# Base URL of the IRINS search page
base_url = "https://irins.org/irins/a/searchc/search"

# List to store all scientists' data
data = []

# Create a cloudscraper instance with a randomized user-agent
scraper = cloudscraper.create_scraper(browser='chrome')

# Install the correct version of chromedriver
chromedriver_autoinstaller.install()

# Set up Selenium WebDriver with a randomized user-agent
options = webdriver.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
]
options.add_argument(f'user-agent={random.choice(user_agents)}')
options.add_argument('--headless')  # Run headless Chrome
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# Function to parse the scientist data from a single page
def parse_list_page(soup):
    scientists = []
    # Find all the divs that contain the scientist information
    scientist_divs = soup.find_all('div', class_='list-product-description product-description-brd')
    for div in scientist_divs:
        name = div.find('strong').text.strip()
        vidwan_id = div.find('b', string='Vidwan-ID : ').next_sibling.strip()
        department = div.find('i', class_='glyphicon glyphicon-object-align-bottom').find_next_sibling(string=True).strip()
        profile_link = div.find('a', string='View Profile')['href']
        # Find the college/building name
        college = div.find('i', class_='fa fa-building').find_next_sibling(string=True).strip()
        scientists.append((name, vidwan_id, department, profile_link, college))
    return scientists

# Function to parse an individual profile page for additional details
def parse_profile_page(url):
    profile_data = {"total_citations": "N/A", "h_index": "N/A", "journal_articles": "N/A"}
    response = scraper.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract total citations
    citations_div = soup.find('span', class_='service-heading black-text small-text', string='Citations')
    if citations_div:
        citations_span = citations_div.find_previous_sibling('span', class_='counter')
        if citations_span:
            profile_data["total_citations"] = citations_span.text.strip()

    # Extract h-index
    h_index_div = soup.find('span', class_='service-heading black-text small-text', string='h-index')
    if h_index_div:
        h_index_span = h_index_div.find_previous_sibling('span', class_='counter')
        if h_index_span:
            profile_data["h_index"] = h_index_span.text.strip()

    # Extract journal articles
    journal_articles_div = soup.find('h6', string='Journal Articles')
    if journal_articles_div:
        journal_articles_span = journal_articles_div.find_previous_sibling('div', class_='p0')
        if journal_articles_span:
            profile_data["journal_articles"] = journal_articles_span.text.strip()

    return profile_data

# Open the initial URL
driver.get(base_url)
time.sleep(random.uniform(1, 2))  # Allow time for the page to load with a random delay

# Select "Exact Search" radio button
exact_search_radio = driver.find_element(By.XPATH, '//input[@name="opt" and @value="exact"]')
exact_search_radio.click()

# Search for the college
search_query = "G L A University"
search_input = driver.find_element(By.ID, "title")
search_input.send_keys(search_query)
search_button = driver.find_element(By.ID, "search")
search_button.click()

# Allow time for the search results to load
time.sleep(random.uniform(2, 4))

# Apply the filters using JavaScript
college_name = "G L A University"
script = f"""
if (document.querySelector('input[name="organisation[]"][value="{college_name}"]')) {{
    document.querySelector('input[name="organisation[]"][value="{college_name}"]').checked = true;
    document.querySelector('form').submit();
}} else {{
    console.log("College input element not found");
}}
"""
driver.execute_script(script)
   # document.querySelector('input[name="wos_sub[]"][value="Engineering and Technology"]').checked = true;
# Allow time for the filters to be applied and results to load
time.sleep(random.uniform(2, 4))

# Loop through each page number to scrape the data
for page_number in range(1, 11):  # Adjust the range as needed to cover all pages
    print(f"Scraping list page: {page_number}")

    if page_number > 1:
        # Click on the pagination link for the next page
        pagination_link = driver.find_element(By.XPATH, f'//a[@id="pagenos" and text()="{page_number}"]')
        pagination_link.click()
        time.sleep(random.uniform(2, 4))  # Allow time for the page to load with a random delay

    # Get the current page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    scientists = parse_list_page(soup)

    for scientist in scientists:
        name, vidwan_id, department, profile_link, college = scientist
        print(f"Scraping profile for {name} (Vidwan ID: {vidwan_id})")
        profile_data = parse_profile_page(profile_link)
        scientist_data = [name, vidwan_id, department, college] + list(profile_data.values())
        data.append(scientist_data)

        # To prevent overwhelming the server, add a delay
        time.sleep(random.uniform(1, 3))  # Random delay to mimic human behavior

# Close the WebDriver
driver.quit()

# Define column names
columns = ["Name", "Vidwan ID", "Department", "College", "Total Citations", "H Index", "Journal Articles"]

# Create a DataFrame and save it to a CSV file
df = pd.DataFrame(data, columns=columns)
df.to_csv('irins_scientists_G L A University.csv', index=False)

print("Data has been scraped")


Scraping list page: 1
Scraping profile for Prof                                                                                ANAND SINGH JALAL (Vidwan ID: 480576)
Scraping profile for Prof                                                                                Kamal Sharma (Vidwan ID: 199451)
Scraping profile for Prof                                                                                Saibal Ray (Vidwan ID: 400377)
Scraping profile for Dr                                                                                Saurabh Gupta (Vidwan ID: 371839)
Scraping profile for Mr                                                                                Aman Sharma (Vidwan ID: 438025)
Scraping profile for Mr                                                                                Shashank Chaturvedi (Vidwan ID: 185768)
Scraping profile for Dr                                                                                Dipak Kumar Das (Vidwan ID: 400360)
Scrapin